## 予備役あるいは非戦闘任務から処分イベントへの移行に関するEHAデータの作成

In [ ]:
import pandas as pd
import numpy as np
import os
import re
import logging

# --- 設定 ---
BASE_DIR = 'https://github.com/naoki-kokaze/British_Warship_Career/tree/main/'
# GraphDBで生成した生のログファイルを参照
INPUT_CSV_PATH = os.path.join(BASE_DIR, 'data', 'presence_log.csv')
# ステップC2の出力ファイル (2種類)
OUTPUT_CSV_ORD = os.path.join(BASE_DIR, 'data', 'eha_dataset_disposal_from_Ordinary.csv')
OUTPUT_CSV_NCD = os.path.join(BASE_DIR, 'data', 'eha_dataset_disposal_from_NonCombat.csv')
LOG_FILE = os.path.join(BASE_DIR, 'log', 'eha_preparation_C.log')

logging.basicConfig(filename=LOG_FILE, level=logging.INFO, filemode='w',
                    format='%(asctime)s - %(levelname)s - %(message)s')
print(f"Log will be saved to: {LOG_FILE}")
print(f"Output CSV 1 (Ordinary) will be saved to: {OUTPUT_CSV_ORD}")
print(f"Output CSV 2 (NonCombat) will be saved to: {OUTPUT_CSV_NCD}")

# --- 時代区分の定義 (仮説C用) ---
PERIOD_PRE_WARRIOR = (1850, 1859)
PERIOD_WARRIOR_ERA = (1860, 1869) # 仮説(i) 延命期
PERIOD_PURGE_ERA = (1870, 1879)   # 仮説(ii) 整理期

# --- ヘルパー関数 ---
def parse_eha_date(date_str):
    if pd.isna(date_str):
        return pd.NaT
    date_str = str(date_str).strip()
    try:
        if re.match(r'^\d{4}$', date_str): # YYYY
            return pd.to_datetime(f"{date_str}-01-01")
        if re.match(r'^\d{4}-\d{2}$', date_str): # YYYY-MM
            return pd.to_datetime(f"{date_str}-01")
        if re.match(r'^\d{4}-\d{2}-\d{2}', date_str): # YYYY-MM-DD...
            return pd.to_datetime(date_str.split('T')[0])
    except Exception as e:
        logging.warning(f"Could not parse date: {date_str}. Error: {e}")
        return pd.NaT
    return pd.NaT

def calculate_duration_years(start_date, end_date):
    if pd.isna(start_date) or pd.isna(end_date):
        return np.nan
    try:
        duration_days = (end_date - start_date).days
        if duration_days <= 0:
             return np.nan
        return duration_days / 365.25
    except Exception:
        return np.nan

def assign_period_flag_C(start_year):
    """仮説C用の時代区分フラグを割り当てる"""
    if pd.isna(start_year):
        return "Other_Pre_1850" # ベースライン
    start_year = int(start_year)
    if PERIOD_PRE_WARRIOR[0] <= start_year <= PERIOD_PRE_WARRIOR[1]:
        return "PreWarrior_1850s"
    if PERIOD_WARRIOR_ERA[0] <= start_year <= PERIOD_WARRIOR_ERA[1]:
        return "WarriorEra_1860s"
    if PERIOD_PURGE_ERA[0] <= start_year <= PERIOD_PURGE_ERA[1]:
        return "PurgeEra_1870s"
    if start_year < 1850:
        return "Other_Pre_1850" # ベースライン
    return "Other_Post_1880"

def assign_category_group_C(category_code):
    """艦種グループを割り当てる (Bと同様)"""
    if pd.isna(category_code):
        return "Unknown"
    code = str(category_code).split('_')[0]
    if code.startswith('SC') or code.startswith('A') or code.startswith('P'):
         return "NewTech_SteamOrArmoured" # ベースライン
    if code in ['SL1', 'SL2', 'SL3', 'SL4']:
        return "Sailing_LineOfBattle"
    if code in ['SF5', 'SC6']:
        return "Sailing_FrigateCorvette"
    if code in ['SSL', 'SBS', 'SCS', 'SM']:
        return "Sailing_Other"
    return "Unknown"

# --- 共通の整形処理を行う関数 ---
def process_eha_subset(df_subset, analysis_status_name):
    """
    フィルタリングされたDataFrame (InOrdinary または InNonCombatDuty) を受け取り、
    EHA変数（duration, event, age, period, category）を作成して返す
    """
    print(f"Processing {len(df_subset)} records for status: {analysis_status_name}")
    logging.info(f"Processing {len(df_subset)} records for status: {analysis_status_name}")

    # 日付データのパース
    df_subset['birth_date_dt'] = df_subset['birth_date'].apply(parse_eha_date)
    df_subset['start_date_dt'] = df_subset['start_date'].apply(parse_eha_date)
    df_subset['end_date_dt'] = df_subset['end_date'].apply(parse_eha_date)

    # 必要な日付データが欠損している行を除外
    df_cleaned = df_subset.dropna(subset=['birth_date_dt', 'start_date_dt', 'end_date_dt']).copy()
    dropped_count = len(df_subset) - len(df_cleaned)
    if dropped_count > 0:
        print(f"Dropped {dropped_count} rows due to missing essential date data.")
        logging.warning(f"Dropped {dropped_count} rows due to missing essential date data.")

    if df_cleaned.empty:
        print(f"No valid records remaining for {analysis_status_name}.")
        logging.error(f"No valid records remaining for {analysis_status_name}.")
        return None

    # EHA変数の作成
    print(f"Creating EHA variables for {analysis_status_name}...")

    # duration: この状態だった期間（年）
    df_cleaned['duration'] = df_cleaned.apply(
        lambda row: calculate_duration_years(row['start_date_dt'], row['end_date_dt']),
        axis=1
    )

    # event: 1 = キャリア終了 (解体 E6_Destruction または 売却 E8_Acquisition)
    disposal_events = ['E6_Destruction', 'E8_Acquisition']
    df_cleaned['event'] = df_cleaned['end_event_type'].apply(
        lambda x: 1 if x in disposal_events else 0
    )

    # age_at_start: この状態が始まった時点の艦齢（年）
    df_cleaned['age_at_start'] = df_cleaned.apply(
        lambda row: calculate_duration_years(row['birth_date_dt'], row['start_date_dt']),
        axis=1
    )

    # period_flag: 時代区分
    df_cleaned['start_year'] = df_cleaned['start_date_dt'].dt.year
    df_cleaned['period_flag'] = df_cleaned['start_year'].apply(assign_period_flag_C)

    # category_group: 艦種グループ
    df_cleaned['category_group'] = df_cleaned['category_code'].apply(assign_category_group_C)

    # 最終クリーニング
    df_final = df_cleaned.dropna(subset=['duration', 'age_at_start', 'category_group', 'period_flag'])
    df_final = df_final[df_final['duration'] > 0]
    df_final = df_final[df_final['category_group'] != 'Unknown']

    print(f"Final dataset for {analysis_status_name} contains {len(df_final)} observations.")
    logging.info(f"Final dataset for {analysis_status_name} contains {len(df_final)} observations.")

    return df_final

# --- メイン処理 ---
def prepare_data_C():
    print("--- Starting EHA Data Preparation (Hypothesis C: Disposal) ---")
    logging.info("--- Starting EHA Data Preparation (Hypothesis C: Disposal) ---")

    # 1. CSVファイルを読み込む
    try:
        print(f"Loading presence log from: {INPUT_CSV_PATH}")
        df = pd.read_csv(INPUT_CSV_PATH)
        logging.info(f"Loaded {len(df)} records from {INPUT_CSV_PATH}")
    except FileNotFoundError as e:
        print(f"ERROR: File not found. {e}")
        logging.error(f"File not found: {e}")
        return
    except Exception as e:
        print(f"ERROR reading CSV files: {e}")
        logging.error(f"Error reading CSV files: {e}")
        return

    # 2. 2つの分析対象をフィルタリング
    df_ordinary = df[df['status'] == 'InOrdinary'].copy()
    df_noncombat = df[df['status'] == 'InNonCombatDuty'].copy()

    if df_ordinary.empty:
        print("Warning: No records found with status 'InOrdinary'.")
        logging.warning("No records found with status 'InOrdinary'.")
    if df_noncombat.empty:
        print("Warning: No records found with status 'InNonCombatDuty'.")
        logging.warning("No records found with status 'InNonCombatDuty'.")

    # 3. それぞれのデータセットを処理
    df_final_ordinary = process_eha_subset(df_ordinary, 'InOrdinary')
    df_final_noncombat = process_eha_subset(df_noncombat, 'InNonCombatDuty')

    # 4. 最終的なCSVに保存
    try:
        os.makedirs(os.path.dirname(OUTPUT_CSV_ORD), exist_ok=True)
        if df_final_ordinary is not None and not df_final_ordinary.empty:
            df_final_ordinary.to_csv(OUTPUT_CSV_ORD, index=False, encoding='utf-8')
            print(f"Successfully saved EHA dataset for 'InOrdinary' to {OUTPUT_CSV_ORD}")
            logging.info(f"Successfully saved EHA dataset for 'InOrdinary' to {OUTPUT_CSV_ORD}")
        else:
            print("No data to save for 'InOrdinary'.")
            logging.warning("No data to save for 'InOrdinary'.")

        if df_final_noncombat is not None and not df_final_noncombat.empty:
            df_final_noncombat.to_csv(OUTPUT_CSV_NCD, index=False, encoding='utf-8')
            print(f"Successfully saved EHA dataset for 'InNonCombatDuty' to {OUTPUT_CSV_NCD}")
            logging.info(f"Successfully saved EHA dataset for 'InNonCombatDuty' to {OUTPUT_CSV_NCD}")
        else:
            print("No data to save for 'InNonCombatDuty'.")
            logging.warning("No data to save for 'InNonCombatDuty'.")

    except Exception as e:
        msg = f"Error saving CSV files: {e}"
        print(msg)
        logging.error(msg)

if __name__ == "__main__":
    # データ整形処理を実行
    prepare_data_C()

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from lifelines import KaplanMeierFitter, CoxPHFitter
import logging
import matplotlib.ticker as mticker
import matplotlib.patches as mpatches

# --- 設定 ---
BASE_DIR = 'https://github.com/naoki-kokaze/British_Warship_Career/tree/main/'
# GraphDBで生成した生のログファイルを参照
INPUT_DIR  = os.path.join(BASE_DIR, 'data')
OUTPUT_DIR = os.path.join(BASE_DIR, 'visualizations')
# ステップC2で生成したCSVファイル (2種類)
INPUT_CSV_ORD = os.path.join(INPUT_DIR, 'eha_dataset_disposal_from_Ordinary.csv')
INPUT_CSV_NCD = os.path.join(INPUT_DIR, 'eha_dataset_disposal_from_NonCombat.csv')

os.makedirs(OUTPUT_DIR, exist_ok=True)

# ログ
logging.basicConfig(filename=LOG_FILE, level=logging.INFO, filemode='w',
                    format='%(asctime)s - %(levelname)s - %(message)s')
print(f"Log will be saved to: {LOG_FILE}")
print(f"Visualization outputs will be saved to: {OUTPUT_DIR}")

# スタイル
sns.set(style="whitegrid")
print("Seaborn style set.")


def stars(p):
    return "***" if p < 0.001 else "**" if p < 0.01 else "*" if p < 0.05 else ""

# --- メイン処理（1ファイル分） ---
def run_eha_analysis_C(input_csv_path, analysis_name_suffix):
    """
    指定されたCSVパスのデータを読み込み、KM と CoxPH（HR図にp値＋2段階帯）を出力
    analysis_name_suffix: "from_Ordinary" / "from_NonCombatDuty" 等
    """
    print(f"\n--- Starting EHA Analysis for: {analysis_name_suffix} ---")
    logging.info(f"--- Starting EHA Analysis for: {analysis_name_suffix} ---")

    # 1) 読み込み
    try:
        print(f"Loading EHA dataset from: {input_csv_path}")
        df = pd.read_csv(input_csv_path)
        logging.info(f"Loaded {len(df)} records from {input_csv_path}")
    except FileNotFoundError as e:
        print(f"ERROR: File not found. {e}")
        logging.error(f"File not found: {e}")
        return
    except Exception as e:
        print(f"ERROR reading CSV files: {e}")
        logging.error(f"Error reading CSV files: {e}")
        return

    # 2) 必須列チェック
    required_cols = ['duration', 'event', 'age_at_start', 'period_flag', 'category_group']
    missing = [c for c in required_cols if c not in df.columns]
    if missing:
        print(f"ERROR: CSV is missing required columns for EHA: {', '.join(missing)}")
        logging.error(f"CSV is missing required columns for EHA: {', '.join(missing)}")
        return

    # 3) クリーニング
    df_eha = df.dropna(subset=['duration', 'age_at_start'])
    df_eha = df_eha[np.isfinite(df_eha['duration']) & np.isfinite(df_eha['age_at_start'])]

    if df_eha.empty:
        print(f"No valid data for EHA ({analysis_name_suffix}) after cleaning NaN/Inf.")
        logging.error(f"No valid data for EHA ({analysis_name_suffix}) after cleaning NaN/Inf.")
        return

    print(f"Using {len(df_eha)} observations for EHA ({analysis_name_suffix}).")

    # 4) KM
    """
    print(f"Generating Kaplan-Meier Plot (Disposal by Period) for {analysis_name_suffix}...")
    logging.info(f"Generating Kaplan-Meier Plot (Disposal by Period) for {analysis_name_suffix}")

    kmf = KaplanMeierFitter()
    plt.figure(figsize=(12, 8))
    ax_km = plt.subplot(111)

    periods_to_plot = ['PreWarrior_1850s', 'WarriorEra_1860s', 'PurgeEra_1870s', 'Other_Post_1880', 'Other_Pre_1850']
    for period in periods_to_plot:
        data = df_eha[df_eha['period_flag'] == period]
        if not data.empty:
            kmf.fit(data['duration'], data['event'], label=f"{period} (N={len(data)})")
            kmf.plot_survival_function(ax=ax_km)

    plt.title(f'Kaplan-Meier: Survival from Disposal (from {analysis_name_suffix})', fontsize=16)
    plt.xlabel(f'Duration in "{analysis_name_suffix}" (Years)', fontsize=12)
    plt.ylabel('Probability of *Not* Being Disposed (Sold/Broken Up)', fontsize=12)
    plt.legend(title='Period Started This Status')
    plt.grid(True, linestyle='--', alpha=0.7)

    km_plot_path = os.path.join(OUTPUT_DIR, f'km_plot_disposal_{analysis_name_suffix}.png')
    try:
        plt.savefig(km_plot_path, dpi=300, bbox_inches='tight')
        print(f"Kaplan-Meier plot (Disposal) saved to {km_plot_path}")
        logging.info(f"Kaplan-Meier plot (Disposal) saved to {km_plot_path}")
    except Exception as e:
        print(f"Error saving KM plot (Disposal): {e}")
        logging.error(f"Error saving KM plot (Disposal): {e}")
    plt.close()
    """

    # 5) CoxPH
    print(f"Running Cox Proportional Hazards Model (Disposal) for {analysis_name_suffix}...")
    logging.info(f"Running Cox Proportional Hazards Model (Disposal) for {analysis_name_suffix}")

    try:
        df_cox = pd.get_dummies(
            df_eha,
            columns=['period_flag', 'category_group'],
            prefix={'period_flag': 'period', 'category_group': 'cat'},
            drop_first=True, dtype=int
        )
    except Exception as e:
        print(f"ERROR creating dummy variables: {e}")
        logging.error(f"ERROR creating dummy variables: {e}")
        return

    period_dummies   = [c for c in df_cox.columns if c.startswith('period_')]
    category_dummies = [c for c in df_cox.columns if c.startswith('cat_')]
    covariates = ['age_at_start'] + period_dummies + category_dummies

    final_cols = ['duration', 'event'] + covariates
    if any(c not in df_cox.columns for c in final_cols):
        missing_in_df = [c for c in final_cols if c not in df_cox.columns]
        print(f"ERROR: Columns missing after dummy creation: {missing_in_df}")
        logging.error(f"Columns missing after dummy creation: {missing_in_df}")
        return

    df_cox_final = df_cox[final_cols].copy()
    df_cox_final = df_cox_final.replace([np.inf, -np.inf], np.nan).dropna()

    if df_cox_final.empty or not covariates:
        print(f"ERROR: No data or no covariates left for CoxPH model. Covariates found: {len(covariates)}")
        logging.error(f"ERROR: No data or no covariates left for CoxPH model. Covariates: {len(covariates)}")
        return

    cph = CoxPHFitter(penalizer=1e-5, l1_ratio=0.0)  # 安定化の弱いRidge
    try:
        cph.fit(df_cox_final, duration_col='duration', event_col='event')

        # サマリ保存
        summary = cph.summary
        print(f"\n--- CoxPH Model Summary (Disposal from {analysis_name_suffix}) ---")
        print(summary)
        logging.info(f"\n--- CoxPH Model Summary (Disposal from {analysis_name_suffix}) ---\n{summary.to_string()}")

        summary_path = os.path.join(OUTPUT_DIR, f'coxph_summary_disposal_{analysis_name_suffix}.csv')
        summary.to_csv(summary_path, encoding='utf-8-sig')
        print(f"Model summary (Disposal) saved to {summary_path}")

        # --- HR図（p値整列＋2段階帯） ---
        plt.figure(figsize=(10, max(12, len(covariates) * 0.5)))
        ax = cph.plot(hazard_ratios=True)
        ax.set_xscale('log')
        ax.axvline(1, ls='--', color='gray', alpha=.8)
        ax.set_xticks([0.25, 0.5, 1, 2, 4])
        ax.get_xaxis().set_major_formatter(mticker.ScalarFormatter())

        # 帯の設定
        sig_color    = '#FFF3B0'  # p<0.05（濃い黄）
        sig_alpha    = 0.55
        marg_color   = '#FFFBE6'  # 0.05<=p<0.07（薄い黄）
        marg_alpha   = 0.9
        p_sig        = 0.05
        p_marg_upper = 0.07

        # yラベル→座標
        yticks  = ax.get_yticks()
        ylabels = [t.get_text() for t in ax.get_yticklabels()]
        label2y = {lab: y for lab, y in zip(ylabels, yticks)}

        # 右外にp値を並べる
        x_min, x_max = ax.get_xlim()
        x_text = x_max * 1.05

        for var, row in summary.iterrows():
            if var not in label2y:
                continue
            y = label2y[var]
            pval = float(row.get('p', 1.0))

            # 2段階ハイライト
            if pval < p_sig:
                ax.axhspan(y - 0.35, y + 0.35, color=sig_color,  alpha=sig_alpha,  zorder=0)
            elif p_sig <= pval < p_marg_upper:
                ax.axhspan(y - 0.35, y + 0.35, color=marg_color, alpha=marg_alpha, zorder=0)

            # p値注記（右外）
            ax.text(x_text, y, f"p = {pval:.3f}{stars(pval)}", va='center', fontsize=10)

        # 注記が切れないように右へ少し拡張
        ax.set_xlim(x_min, x_text * 1.12)

        # 帯の凡例を追加
        patch_sig  = mpatches.Patch(facecolor=sig_color,  alpha=sig_alpha,  label=f'p < {p_sig}')
        patch_marg = mpatches.Patch(facecolor=marg_color, alpha=marg_alpha, label=f'{p_sig} ≤ p < {p_marg_upper}')
        handles, labels = ax.get_legend_handles_labels()
        ax.legend(handles + [patch_sig, patch_marg],
                  labels + [patch_sig.get_label(), patch_marg.get_label()],
                  loc='upper right', title='Significance')

        plt.title(f'CoxPH Model: Hazard Ratios (exp(coef)) with p-values\n(from {analysis_name_suffix})', fontsize=14)
        plt.xlabel('Hazard Ratio (exp(coef))', fontsize=12)
        plt.grid(True, linestyle='--', alpha=0.7)

        out_svg = os.path.join(OUTPUT_DIR, f'coxph_plot_disposal_{analysis_name_suffix}_with_p_highlighted.svg')
        plt.savefig(out_svg, format='svg', bbox_inches='tight')

        print(f"Model coefficients plot (with p-values) saved to {out_svg}")
        plt.close()

    except Exception as e:
        print(f"ERROR: CoxPH model fitting failed: {e}")
        logging.exception(f"CoxPH model fitting failed: {e}")

    print(f"--- EHA Analysis for {analysis_name_suffix} Finished ---")
    logging.info(f"--- EHA Analysis for {analysis_name_suffix} Finished ---")

# --- エントリポイント ---
if __name__ == "__main__":
    # 2つの分析を順番に実行
    run_eha_analysis_C(INPUT_CSV_ORD, "from_Ordinary")
    run_eha_analysis_C(INPUT_CSV_NCD, "from_NonCombatDuty")

    print("\n--- All EHA-C Analyses Finished ---")
    logging.info("--- All EHA-C Analyses Finished ---")


## eha_dataset_retirement.csv 作成用のコードセル

In [ ]:
import pandas as pd
import numpy as np
import os
import re
import logging

# --- 設定 ---
BASE_DIR = 'https://github.com/naoki-kokaze/British_Warship_Career/tree/main/'
# ステップ1で生成した生のログファイルを参照
INPUT_CSV_PATH = os.path.join(BASE_DIR, 'data', 'presence_log.csv')
# ステップD2の出力ファイル
OUTPUT_CSV_PATH = os.path.join(BASE_DIR, 'data', 'eha_dataset_retirement.csv')
LOG_FILE = os.path.join(BASE_DIR, 'log', 'eha_preparation_D.log')

logging.basicConfig(filename=LOG_FILE, level=logging.INFO, filemode='w',
                    format='%(asctime)s - %(levelname)s - %(message)s')
print(f"Log will be saved to: {LOG_FILE}")
print(f"Output CSV will be saved to: {OUTPUT_CSV_PATH}")

# --- 時代区分の定義 (仮説D用) ---
# (EHA-Cと同じ定義を使用)
PERIOD_PRE_WARRIOR = (1850, 1859)
PERIOD_WARRIOR_ERA = (1860, 1869)
PERIOD_PURGE_ERA = (1870, 1879)

# --- ヘルパー関数 (EHA-Cと同様) ---
def parse_eha_date(date_str):
    if pd.isna(date_str):
        return pd.NaT
    date_str = str(date_str).strip()
    try:
        if re.match(r'^\d{4}$', date_str): # YYYY
            return pd.to_datetime(f"{date_str}-01-01")
        if re.match(r'^\d{4}-\d{2}$', date_str): # YYYY-MM
            return pd.to_datetime(f"{date_str}-01")
        if re.match(r'^\d{4}-\d{2}-\d{2}', date_str): # YYYY-MM-DD...
            return pd.to_datetime(date_str.split('T')[0])
    except Exception as e:
        logging.warning(f"Could not parse date: {date_str}. Error: {e}")
        return pd.NaT
    return pd.NaT

def calculate_duration_years(start_date, end_date):
    if pd.isna(start_date) or pd.isna(end_date):
        return np.nan
    try:
        duration_days = (end_date - start_date).days
        if duration_days <= 0:
             return np.nan
        return duration_days / 365.25
    except Exception:
        return np.nan

def assign_period_flag_D(start_year):
    """仮説D用の時代区分フラグを割り当てる"""
    if pd.isna(start_year):
        return "Other_Pre_1850" # ベースライン
    start_year = int(start_year)
    if PERIOD_PRE_WARRIOR[0] <= start_year <= PERIOD_PRE_WARRIOR[1]:
        return "PreWarrior_1850s"
    if PERIOD_WARRIOR_ERA[0] <= start_year <= PERIOD_WARRIOR_ERA[1]:
        return "WarriorEra_1860s"
    if PERIOD_PURGE_ERA[0] <= start_year <= PERIOD_PURGE_ERA[1]:
        return "PurgeEra_1870s"
    if start_year < 1850:
        return "Other_Pre_1850" # ベースライン
    return "Other_Post_1880"

def assign_category_group_D(category_code):
    """艦種グループを割り当てる (B/Cと同様)"""
    if pd.isna(category_code):
        return "Unknown"
    code = str(category_code).split('_')[0]
    if code.startswith('SC') or code.startswith('A') or code.startswith('P'):
         return "NewTech_SteamOrArmoured" # ベースライン
    if code in ['SL1', 'SL2', 'SL3', 'SL4']:
        return "Sailing_LineOfBattle"
    if code in ['SF5', 'SC6']:
        return "Sailing_FrigateCorvette"
    if code in ['SSL', 'SBS', 'SCS', 'SM']:
        return "Sailing_Other"
    return "Unknown"

# --- メイン処理 ---
def prepare_data_D():
    print("--- Starting EHA Data Preparation (Hypothesis D: Retirement/Competing Risks) ---")
    logging.info("--- Starting EHA Data Preparation (Hypothesis D) ---")

    # 1. CSVファイルを読み込む
    try:
        print(f"Loading presence log from: {INPUT_CSV_PATH}")
        df = pd.read_csv(INPUT_CSV_PATH)
        logging.info(f"Loaded {len(df)} records from {INPUT_CSV_PATH}")
    except FileNotFoundError as e:
        print(f"ERROR: File not found. {e}")
        logging.error(f"File not found: {e}")
        return
    except Exception as e:
        print(f"ERROR reading CSV files: {e}")
        logging.error(f"Error reading CSV files: {e}")
        return

    # 2. 艦船ごとにソートし、次の状態（next_status）カラムを作成
    print("Sorting data and identifying next status...")
    logging.info("Sorting data and identifying next status...")
    df['start_date_dt'] = df['start_date'].apply(parse_eha_date)
    df_sorted = df.dropna(subset=['start_date_dt']).sort_values(by=['ship', 'start_date_dt'])
    df_sorted['next_status'] = df_sorted.groupby('ship')['status'].shift(-1)

    # 3. 分析対象のフィルタリング (現役の期間)
    # ★★★ 分析対象を変更 ★★★
    analysis_statuses = ['InCommission']
    df_analysis = df_sorted[df_sorted['status'].isin(analysis_statuses)].copy()

    if df_analysis.empty:
        print("ERROR: No records found with status 'InCommission'.")
        logging.error("No records found with status 'InCommission'.")
        return
    print(f"Filtered {len(df_analysis)} 'InCommission' records.")
    logging.info(f"Filtered {len(df_analysis)} 'InCommission' records.")

    # 4. 日付データのパース
    print("Parsing (remaining) dates...")
    logging.info("Parsing (remaining) dates...")
    df_analysis['birth_date_dt'] = df_analysis['birth_date'].apply(parse_eha_date)
    df_analysis['end_date_dt'] = df_analysis['end_date'].apply(parse_eha_date)

    # 必要な日付データが欠損している行を除外
    df_cleaned = df_analysis.dropna(subset=['birth_date_dt', 'start_date_dt', 'end_date_dt']).copy()
    dropped_count = len(df_analysis) - len(df_cleaned)
    if dropped_count > 0:
        print(f"Dropped {dropped_count} rows due to missing essential date data.")
        logging.warning(f"Dropped {dropped_count} rows due to missing essential date data.")

    if df_cleaned.empty:
        print("ERROR: No valid records remaining after date cleaning.")
        logging.error("No valid records remaining after date cleaning.")
        return

    # 5. EHA変数の作成
    print("Creating EHA variables (duration, events, age_at_start, period_flag, category_group)...")
    logging.info("Creating EHA variables...")

    # duration: 現役だった期間（年）
    df_cleaned['duration'] = df_cleaned.apply(
        lambda row: calculate_duration_years(row['start_date_dt'], row['end_date_dt']),
        axis=1
    )

    # ★★★ 競合リスク（Competing Risks）のイベント列を作成 ★★★

    # イベントA: 退役 (→ InOrdinary)
    df_cleaned['event_A_Retired'] = df_cleaned['next_status'].apply(
        lambda x: 1 if x == 'InOrdinary' else 0
    )

    # イベントB: 転用 (→ InNonCombatDuty)
    df_cleaned['event_B_Converted'] = df_cleaned['next_status'].apply(
        lambda x: 1 if x == 'InNonCombatDuty' else 0
    )

    # イベントC: 処分 (→ キャリア終了, next_statusがNaN)
    df_cleaned['event_C_Disposed'] = df_cleaned['next_status'].apply(
        lambda x: 1 if pd.isna(x) else 0
    )

    # (参考) 総合イベント列 (いずれかが発生した場合 = 1)
    df_cleaned['event_Any'] = (df_cleaned['event_A_Retired'] |
                             df_cleaned['event_B_Converted'] |
                             df_cleaned['event_C_Disposed'])

    # ----------------------------------------------------

    # age_at_start: 現役が始まった時点の艦齢（年）
    df_cleaned['age_at_start'] = df_cleaned.apply(
        lambda row: calculate_duration_years(row['birth_date_dt'], row['start_date_dt']),
        axis=1
    )

    # period_flag: 時代区分
    df_cleaned['start_year'] = df_cleaned['start_date_dt'].dt.year
    df_cleaned['period_flag'] = df_cleaned['start_year'].apply(assign_period_flag_D)

    # category_group: 艦種グループ
    df_cleaned['category_group'] = df_cleaned['category_code'].apply(assign_category_group_D)

    # 6. 最終クリーニング
    df_final = df_cleaned.dropna(subset=['duration', 'age_at_start', 'category_group', 'period_flag'])
    df_final = df_final[df_final['duration'] > 0]
    df_final = df_final[df_final['category_group'] != 'Unknown']

    # 少なくとも1つのイベントが発生している（打ち切りでない）行が対象
    df_final = df_final[df_final['event_Any'] == 1]

    print(f"Final dataset contains {len(df_final)} observations for EHA (Hypothesis D).")
    logging.info(f"Final dataset contains {len(df_final)} observations for EHA (Hypothesis D).")

    # 7. 最終的なCSVに保存
    try:
        os.makedirs(os.path.dirname(OUTPUT_CSV_PATH), exist_ok=True)
        df_final.to_csv(OUTPUT_CSV_PATH, index=False, encoding='utf-8')

        print(f"Successfully saved EHA dataset for Retirement/Competing Risks to {OUTPUT_CSV_PATH}")
        logging.info(f"Successfully saved EHA dataset for Retirement/Competing Risks to {OUTPUT_CSV_PATH}")
    except Exception as e:
        msg = f"Error saving CSV file to {OUTPUT_CSV_PATH}: {e}"
        print(msg)
        logging.error(msg)

if __name__ == "__main__":
    # データ整形処理を実行
    prepare_data_D()

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from lifelines import CoxPHFitter
import logging
import matplotlib.ticker as mticker
import matplotlib.patches as mpatches

# --- 設定 ---
BASE_DIR = 'https://github.com/naoki-kokaze/British_Warship_Career/tree/main/'
INPUT_CSV_PATH = os.path.join(BASE_DIR, 'data', 'eha_dataset_retirement.csv')
OUTPUT_DIR = os.path.join(BASE_DIR, 'visualizations')
LOG_FILE   = os.path.join(BASE_DIR, 'log', 'eha_analysis_D.log')

os.makedirs(OUTPUT_DIR, exist_ok=True)

logging.basicConfig(filename=LOG_FILE, level=logging.INFO, filemode='w',
                    format='%(asctime)s - %(levelname)s - %(message)s')
print(f"Log will be saved to: {LOG_FILE}")
print(f"Visualization outputs will be saved to: {OUTPUT_DIR}")

sns.set(style="whitegrid")

def stars(p):
    """p値に応じたスター表記"""
    return "***" if p < 0.001 else "**" if p < 0.01 else "*" if p < 0.05 else ""

def run_competing_risk_model(df_eha, event_column_name, analysis_name_suffix):
    """指定された event 列で CoxPH を実行し、HR 図に p値および有意・周辺有意のハイライトを付けて保存"""

    print(f"\n--- Running CoxPH Model for Event: {event_column_name} ({analysis_name_suffix}) ---")
    logging.info(f"--- Running CoxPH Model for Event: {event_column_name} ({analysis_name_suffix}) ---")

    # ダミー化
    try:
        df_cox = pd.get_dummies(
            df_eha,
            columns=['period_flag', 'category_group'],
            prefix={'period_flag': 'period', 'category_group': 'cat'},
            drop_first=True, dtype=int
        )
    except Exception as e:
        print(f"ERROR creating dummy variables: {e}")
        logging.error(f"ERROR creating dummy variables: {e}")
        return

    period_dummies   = [c for c in df_cox.columns if c.startswith('period_')]
    category_dummies = [c for c in df_cox.columns if c.startswith('cat_')]
    covariates = ['age_at_start'] + period_dummies + category_dummies

    # event列を共通名に変更
    need_cols = ['duration', event_column_name] + covariates
    missing = [c for c in need_cols if c not in df_cox.columns]
    if missing:
        print(f"ERROR: Missing columns for CoxPH: {missing}")
        logging.error(f"Missing columns for CoxPH: {missing}")
        return

    df_cox_final = df_cox[need_cols].rename(columns={event_column_name: 'event'}).copy()
    df_cox_final = df_cox_final.replace([np.inf, -np.inf], np.nan).dropna()

    if df_cox_final.empty or not covariates:
        print("ERROR: No data or no covariates left for CoxPH.")
        logging.error("No data or no covariates left for CoxPH.")
        return

    if df_cox_final['event'].sum() == 0:
        print(f"Warning: No events=1 for {event_column_name}. Skipping.")
        logging.warning(f"No events=1 for {event_column_name}. Skipping.")
        return

    print(f"Running CoxPH model on {len(df_cox_final)} observations with {len(covariates)} covariates.")
    logging.info(f"Running CoxPH model on {len(df_cox_final)} observations with {len(covariates)} covariates.")

    cph = CoxPHFitter(penalizer=1e-5, l1_ratio=0.0)
    try:
        cph.fit(df_cox_final, duration_col='duration', event_col='event')
        summary = cph.summary

        # --- 結果保存 ---
        summary_path = os.path.join(OUTPUT_DIR, f'coxph_summary_retirement_{analysis_name_suffix}.csv')
        summary.to_csv(summary_path, encoding='utf-8-sig')
        print(f"Model summary saved to {summary_path}")

        # --- 図描画 ---
        plt.figure(figsize=(10, max(12, len(covariates) * 0.5)))
        ax = cph.plot(hazard_ratios=True)
        ax.set_xscale('log')
        ax.axvline(1, ls='--', color='gray', alpha=0.8)
        ax.set_xticks([0.25, 0.5, 1, 2, 4])
        ax.get_xaxis().set_major_formatter(mticker.ScalarFormatter())

        # テキスト配置位置
        x_min, x_max = ax.get_xlim()
        x_text = x_max * 1.05

        # y座標対応
        yticks = ax.get_yticks()
        ylabels = [t.get_text() for t in ax.get_yticklabels()]
        label2y = {lab: y for lab, y in zip(ylabels, yticks)}

        # --- ハイライト設定 ---
        color_sig = '#FFF176'     # 濃い黄色（p<0.05）
        color_marg = '#FFF9C4'    # 薄い黄色（0.05<=p<0.07）
        alpha_band = 0.55

        for var, row in summary.iterrows():
            if var in label2y:
                y = label2y[var]
                pval = float(row.get('p', 1.0))

                # 有意または周辺有意なら帯を描画
                if pval < 0.05:
                    ax.axhspan(y - 0.35, y + 0.35, color=color_sig, alpha=alpha_band, zorder=0)
                elif 0.05 <= pval < 0.07:
                    ax.axhspan(y - 0.35, y + 0.35, color=color_marg, alpha=alpha_band, zorder=0)

                # p値テキスト
                ax.text(x_text, y, f"p = {pval:.3f}{stars(pval)}", va='center', fontsize=10)

        # テキスト分の右余白
        ax.set_xlim(x_min, x_text * 1.12)

        # 凡例追加
        patch_sig = mpatches.Patch(facecolor=color_sig, alpha=alpha_band, label='p < 0.05')
        patch_marg = mpatches.Patch(facecolor=color_marg, alpha=alpha_band, label='0.05 ≤ p < 0.07')
        handles, labels = ax.get_legend_handles_labels()
        ax.legend(handles + [patch_sig, patch_marg],
                  labels + [patch_sig.get_label(), patch_marg.get_label()],
                  loc='upper right', title='Significance')

        plt.title(f'CoxPH Model: Hazard Ratios (exp(coef)) with Significance Bands\n({analysis_name_suffix})', fontsize=14)
        plt.xlabel('Hazard Ratio (exp(coef))', fontsize=12)
        plt.grid(True, linestyle='--', alpha=0.7)

        out_svg = os.path.join(OUTPUT_DIR, f'coxph_plot_retirement_{analysis_name_suffix}_with_p_highlighted.svg')
        plt.savefig(out_svg, format='svg', bbox_inches='tight')

        print(f"Model coefficients plot saved to {out_svg}")
        logging.info(f"Model coefficients plot saved to {out_svg}")
        plt.close()

    except Exception as e:
        print(f"ERROR: CoxPH model fitting failed for {analysis_name_suffix}: {e}")
        logging.exception(f"CoxPH model fitting failed for {analysis_name_suffix}: {e}")

def run_eha_analysis_D():
    print("--- Starting EHA Analysis (Hypothesis D: Retirement/Competing Risks) ---")
    logging.info("--- Starting EHA Analysis (Hypothesis D) ---")

    try:
        df_eha_base = pd.read_csv(INPUT_CSV_PATH)
        logging.info(f"Loaded {len(df_eha_base)} records from {INPUT_CSV_PATH}")
    except Exception as e:
        print(f"ERROR loading file: {e}")
        return

    # 各イベントを順に分析
    for ev, name in [
        ('event_A_Retired', 'A_Retired_to_Ordinary'),
        ('event_B_Converted', 'B_Converted_to_NonCombat'),
        ('event_C_Disposed', 'C_Disposed_from_Commission')
    ]:
        if ev in df_eha_base.columns:
            run_competing_risk_model(df_eha_base, ev, name)
        else:
            print(f"Column {ev} not found. Skipping.")

    print("\n--- All EHA-D Analyses Finished ---")
    logging.info("--- All EHA-D Analyses Finished ---")

if __name__ == "__main__":
    run_eha_analysis_D()
